In [10]:
import os
import dotenv
import pymongo
import requests
import datetime
import threading
from functions import *
from flask import abort
from flask import Flask
from flask import request
from flask import session
from bson import json_util
from flask import redirect
from functools import wraps
from flask import render_template
from flask.wrappers import Response
from werkzeug.datastructures import ImmutableMultiDict

dotenv.load_dotenv()
app = Flask(__name__)
VERIFY_TOKEN = 'cashwha'
MONGO_CLIENT = pymongo.MongoClient(os.getenv('MONGO_SRV'))
WHATSAPP_CONTACTS_COL = MONGO_CLIENT["namma_yatri"]["whatsapp_contacts"]
RIDES_COL = MONGO_CLIENT["namma_yatri"]["rides"]
DRIVERS_COL = MONGO_CLIENT["namma_yatri"]["drivers"]
whatsapp_account = {
    'ACCESS_TOKEN': '',
    'FROM_PHONE_NUMBER_ID': os.getenv('WHATSAPP_FROM_PHONE_NUMBER_ID'),
    'WABA_ID': os.getenv('WHATSAPP_WABA_ID'),
    'FROM_PHONE_NUMBER': os.getenv('WHATSAPP_FROM_PHONE_NUMBER'),
    'VERIFY_TOKEN': os.getenv('WHATSAPP_VERIFY_TOKEN')
}


In [11]:
def incoming_message(contact: dict, message: dict):
    msg = Message(whatsapp_account)
    msg.to = contact.get('number')
    booking_status = contact.get('booking_status', {})
    if message.get('content_type') == 'location':
        if booking_status.get('value') == 'awaiting from location':
            msg.set_message('Thank you please send your to location.')
            msg.send()
            booking_status['value'] = 'awaiting to location'
            booking_status['from'] = message['body']
            WHATSAPP_CONTACTS_COL.update_one(
                {
                    '_id': contact.get('_id')
                },
                {
                    '$set': {
                        'booking_status': booking_status
                    }
                }
            )
            return None # The user sends a from location and the server requests to send a to location
        if booking_status.get('value') == 'awaiting to location':
            msg.set_message('Thank you for using Namma Yatri.\nYour ride has been scheduled and you will be notified once the ride is alotted')
            msg.send()
            booking_status['value'] = 'ride sheduled'
            booking_status['to'] = message['body']
            WHATSAPP_CONTACTS_COL.update_one(
                {
                    '_id': contact.get('_id')
                },
                {
                    '$set': {
                        'booking_status': booking_status
                    }
                }
            )
            # make_order()
            return None # The user sends the to location and the server initiates the order
            # The order is sent back to the customer and sent to the drivers pool
        msg.set_message('Kindly initiate the ride before sending your location.')
        msg.send()
        return None # you have to initialise the order first to send your location
    if message.get('content_type') == 'interactive':
        message_body = message.get('body', {}).get('list_reply', {}).get('title')
        if message_body == 'Book a Ride':
            msg.set_message('Please Send Your current Location to Book a Ride')
            msg.send()
            booking_status['value'] = 'awaiting from location'
            WHATSAPP_CONTACTS_COL.update_one(
                {
                    '_id': contact.get('_id')
                },
                {
                    '$set': {
                        'booking_status': booking_status
                    }
                }
            )
            return None # the server requests for the current location
        if message_body == 'Customer Care':
            msg.set_message('You can contact Customer Care on call via +91 94885 60252\n\nThank you for using Namma Yatri. Have a nice day.')
            msg.send()
            return None
        if message_body == 'Reset':
            booking_status = {'value': 0}
            WHATSAPP_CONTACTS_COL.update_one(
                {
                    '_id': contact.get('_id')
                },
                {
                    '$set': {
                        'booking_status': booking_status
                    }
                }
            )
            msg.set_message('Thank You your data has been resetted.')
            msg.send()
            message['content_type'] = 'text'
            msg = Message(whatsapp_account)
            msg.to = contact.get('number')
    if message.get('content_type') == 'text':
        msg.set_list(
            'Hello Welcome to Namma Yatri.', 
            'Option', 
            [
                {
                    'section_title': 'Please Select an Option',
                    'body': [
                        {
                            'id': '1',
                            'title': 'Book a Ride'
                        },
                        {
                            'id': '2',
                            'title': 'Customer Care'
                        },
                        {
                            'id': '3',
                            'title': 'Reset'
                        }
                    ]
                }
            ]
        )
        response = msg.send()
        return response # The user sent a unknown message so start with a new conversation
    return None

In [12]:
contact = WHATSAPP_CONTACTS_COL.find_one({
    'from_number': '918610487682',
    'number': '919488560252'
})
message = {
    'content_type': 'text',
    'body': 'Hii'
}
# message = {
#     'content_type': 'location',
#     'body': {
#         'North': '234567890',
#         'South': '1234567890'
#     }
# }
incoming_message(contact, message)

{'messaging_product': 'whatsapp',
 'contacts': [{'input': '919488560252', 'wa_id': '919488560252'}],
 'messages': [{'id': 'wamid.HBgMOTE5NDg4NTYwMjUyFQIAERgSMzU3MTQzRTc3MDJFRkZCNDY2AA=='}]}

In [5]:
# arr = ['919341288630', '919842691115', '919003207077', '919025395730', '919566130005', '917395994313' , '919843362266', '919244555599', '917010099029', '919442249206', '916379911489', '919894186988', '919003434031', '918073043250', '919488560252']
arr = ['919488560252', '918667762825']
for i in arr:
    msg = Message(whatsapp_account)
    msg.to = i
    msg.set_message('Hi thank you for opting out to Namma Yatri Hackathon project.\n\nDue to server maintanence down we might be unable to process your request, Sorry for the inconvinience if caused.\n\nYou may start messaging now for any further queries contact Team leader Lamin via laminmuhammed@gmail.com')
    print(msg.send())

{'messaging_product': 'whatsapp', 'contacts': [{'input': '919488560252', 'wa_id': '919488560252'}], 'messages': [{'id': 'wamid.HBgMOTE5NDg4NTYwMjUyFQIAERgSQTNDOTdGRjk1RERGN0VCMUQ3AA=='}]}
{'messaging_product': 'whatsapp', 'contacts': [{'input': '918667762825', 'wa_id': '918667762825'}], 'messages': [{'id': 'wamid.HBgMOTE4NjY3NzYyODI1FQIAERgSMkExMzM1OTMzNURGRkQ3MUY4AA=='}]}
